### Deep Learning Model Evaluation

In [3]:
# importing datasets using chunking

import pandas as pd

df_chunk = pd.read_csv('../datasets/titanic.csv', chunksize=10)

# errors because df_chunk is a generator
# df_chunk['Age']

C = 0
for chunk in df_chunk:
    C += 1
    print(C)
    print(chunk['Age'])

1
0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64
2
10     4.0
11    58.0
12    20.0
13    39.0
14    14.0
15    55.0
16     2.0
17     NaN
18    31.0
19     NaN
Name: Age, dtype: float64
3
20    35.0
21    34.0
22    15.0
23    28.0
24     8.0
25    38.0
26     NaN
27    19.0
28     NaN
29     NaN
Name: Age, dtype: float64
4
30    40.0
31     NaN
32     NaN
33    66.0
34    28.0
35    42.0
36     NaN
37    21.0
38    18.0
39    14.0
Name: Age, dtype: float64
5
40    40.0
41    27.0
42     NaN
43     3.0
44    19.0
45     NaN
46     NaN
47     NaN
48     NaN
49    18.0
Name: Age, dtype: float64
6
50     7.0
51    21.0
52    49.0
53    29.0
54    65.0
55     NaN
56    21.0
57    28.5
58     5.0
59    11.0
Name: Age, dtype: float64
7
60    22.0
61    38.0
62    45.0
63     4.0
64     NaN
65     NaN
66    29.0
67    19.0
68    17.0
69    26.0
Name: Age, dtype: float64
8
70    32.00
71    16.00
72    21.00
73    

78
770    24.0
771    48.0
772    57.0
773     NaN
774    54.0
775    18.0
776     NaN
777     5.0
778     NaN
779    43.0
Name: Age, dtype: float64
79
780    13.0
781    17.0
782    29.0
783     NaN
784    25.0
785    25.0
786    18.0
787     8.0
788     1.0
789    46.0
Name: Age, dtype: float64
80
790     NaN
791    16.0
792     NaN
793     NaN
794    25.0
795    39.0
796    49.0
797    31.0
798    30.0
799    30.0
Name: Age, dtype: float64
81
800    34.00
801    31.00
802    11.00
803     0.42
804    27.00
805    31.00
806    39.00
807    18.00
808    39.00
809    33.00
Name: Age, dtype: float64
82
810    26.0
811    39.0
812    35.0
813     6.0
814    30.5
815     NaN
816    23.0
817    31.0
818    43.0
819    10.0
Name: Age, dtype: float64
83
820    52.0
821    27.0
822    38.0
823    27.0
824     2.0
825     NaN
826     NaN
827     1.0
828     NaN
829    62.0
Name: Age, dtype: float64
84
830    15.00
831     0.83
832      NaN
833    23.00
834    18.00
835    39.00
836    21.00
83

In [4]:
# create a custom generator 
def data_generator(chunk):
    for hunk in chunk:
        yield (hunk['Age'], hunk['Sex'])

df_chunk_titanic = pd.read_csv('../datasets/titanic.csv', chunksize=10)

G = data_generator(df_chunk_titanic)

print(next(G))
print(next(G))

(0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64, 0      male
1    female
2    female
3    female
4      male
5      male
6      male
7      male
8    female
9    female
Name: Sex, dtype: object)
(10     4.0
11    58.0
12    20.0
13    39.0
14    14.0
15    55.0
16     2.0
17     NaN
18    31.0
19     NaN
Name: Age, dtype: float64, 10    female
11    female
12      male
13      male
14    female
15    female
16      male
17      male
18    female
19    female
Name: Sex, dtype: object)


Using generators is useful if say, you have a csv file for image training but the data column is a url. If we preloaded all the images into $x_{train}$, we would run into memory limits. So we can use a generator to fetch each image as the model iterates through the data on the fly

In [5]:
# Milad's example

import numpy as np

def data_gen(df, batch_size):
    while True:
        x_batch = np.zeros((batch_size, 3, 224, 224))
        ## Assume we have two classes (two labels)
        y_batch = np.zeros((batch_size, 1))
        for j in range(len(df['url']/batch_size)):
            b = 0
            for m, k in zip(df['url'].values[j*batch_size:(j+1)*batch_size], df['class'].values[j*batch_size:(j+1)*batch_size]):
                x_batch[b] = m
                y_batch[b] = k
                b += 1
            yield (x_batch, y_batch)
            
model.fit_generator(generator=data_gen(df_train, batch_size=batch_size), steps_per_epoch=len(df_train) // batch_size, epochs=epochs)

NameError: name 'model' is not defined

In [6]:
# from markdown lesson

def batch_generator(df, batch_size, path_tiles, num_classes):
    """This generator use a pandas DataFrame to read images (df.tile_name) from disk.
    """
    N = df.shape[0]
    while True:
        for start in range(0, N, batch_size):
            x_batch = []
            y_batch = []
            end = min(start + batch_size, N)
            df_tmp = df[start:end]
            ids_batch = df_tmp.tile_name
            for id in ids_batch:
                img = cv2.imread(path_tiles+'/{}'.format(id))
                # [0] since duplicated names
                labelname=df_tmp['y'][df_tmp.tile_name == id].values[0]  
                labelname=np.asscalar(labelname)
                x_batch.append(img)
                y_batch.append(labelname)
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = utils.np_utils.to_categorical(y_batch, num_classes)
            yield (x_batch, y_batch)

model.fit_generator(generator=batch_generator(df_train,
                                              batch_size=batch_size,
                                              path_tiles=path_tiles,
                                              num_classes=num_classes),
                    steps_per_epoch=len(df_train) // batch_size,
                    epochs=epochs)

NameError: name 'model' is not defined

### Data Augmentation

Data augmentation is not when you go to the data surgeon to get a bigger data set, but it's kind of close. It's when you apply transforms on the original dataset to create more variation and thus more data for the model to train on. For example, in image recognition we can crop the images, change the lighting, zoom, or tilt the image to simulate other kinds of image inputs that our feature might appear on. Thus instead of going out and capturing a toucan and getting it to cooperate with you in a photography studio setup, you can just take existing toucan pictures and vary the brightness and hue, etc.

### Transfer learning

In practice, very few people train a Convolution network from scratch (with random initialization) because it is rare to get a large enough dataset. So using pretrained network weights as initializations or a "fixed feature extractor" helps in solving most of the problems in hand.

Very deep networks are expensive to train. The most complex models take weeks to train using hundres of machines equipped with expensive GPUs.

Determining the topology/flavour/training method/hyper parameters for deep learning is a black art with not much theory to guide you.

So we need transfer learning.

(One example: Freeze the parameters of a CNN, and use its output as input for an MLP which is being trained.)

In [14]:
from tensorflow.keras import applications, optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

base_model = applications.vgg16.VGG16(include_top=False, weights='imagenet')


i=0
for layer in base_model.layers:
    layer.trainable = False
    i = i+1
    print(i,layer.name)


x = base_model.output
x = Dense(128, activation='sigmoid')(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
predictions = Dense(10, activation='softmax')(x)



model = Model(inputs=base_model.input, outputs=predictions)

model.compile(loss="categorical_crossentropy", optimizer=optimizers.SGD(lr=0.001, momentum=0.9),metrics=["accuracy"])

model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=10,
        callbacks = callbacks_list,
        validation_data = validation_generator,
        validation_steps=20
        )

1 input_3
2 block1_conv1
3 block1_conv2
4 block1_pool
5 block2_conv1
6 block2_conv2
7 block2_pool
8 block3_conv1
9 block3_conv2
10 block3_conv3
11 block3_pool
12 block4_conv1
13 block4_conv2
14 block4_conv3
15 block4_pool
16 block5_conv1
17 block5_conv2
18 block5_conv3
19 block5_pool


NameError: name 'train_generator' is not defined